In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,PolynomialFeatures, MinMaxScaler
from sklearn.impute import SimpleImputer
from scipy.stats import skew
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [74]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')   

df_train.head()

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


In [75]:
df_test.head()

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef
0,48,80851.txt,https://www.iaee.org,19.0,NaN,12.0,0.0,org,NaN,0.079963,...,1.0,0.0,0.0,1.0,NaN,NaN,13.0,194.0,NaN,65.0
1,68,mw130480.txt,http://www.iran-edi.com,22.0,NaN,16.0,NaN,NaN,0.625000,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,2.0,0.0,0.0,1.0
2,76,400382.txt,https://www.bistum-chur.ch,25.0,www.bistum-chur.ch,18.0,0.0,ch,0.636364,0.004983,...,NaN,NaN,0.0,1.0,5.0,12.0,18.0,193.0,NaN,196.0
3,155,625297.txt,https://www.numberthreebath.com,30.0,NaN,23.0,NaN,com,1.000000,0.522907,...,0.0,0.0,NaN,NaN,NaN,1.0,10.0,12.0,0.0,11.0
4,167,8123642.txt,https://ipfs.litnet.work/ipfs/bafybeib5jvxytzb...,100.0,ipfs.litnet.work,NaN,0.0,work,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [76]:
X = df_train.drop(columns=['id'])
y = df_train['id']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
label_encoders = {}

X_combined = pd.concat([X_train, X_val])

for col in categorical_columns:
    encoder = LabelEncoder()
    X_combined[col] = encoder.fit_transform(X_combined[col])
    label_encoders[col] = encoder

X_train = X_combined.iloc[:len(X_train)]
X_val = X_combined.iloc[len(X_train):]

print("Ukuran dataset:")
print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("y_train:", y_train.shape)
print("y_val:", y_val.shape)

Ukuran dataset:
X_train: (112323, 55)
X_val: (28081, 55)
y_train: (112323,)
y_val: (28081,)


In [77]:
from sklearn.impute import SimpleImputer

# Kolom numerik yang akan diimputasi
num_cols = df_train.select_dtypes(include=['int64', 'float64']).columns

# Imputasi dengan median
imputer = SimpleImputer(strategy='median')
df_train[num_cols] = imputer.fit_transform(df_train[num_cols])

# Imputasi kolom kategorikal dengan modus
cat_cols = df_train.select_dtypes(include=['object']).columns
for col in cat_cols:
    df_train[col] = df_train[col].fillna(df_train[col].mode()[0])

# Check if 'sbytes' and 'dbytes' columns exist before filtering
if 'sbytes' in df_train.columns and 'dbytes' in df_train.columns:
    df_cleaned = df_train[(df_train['sbytes'] != 0) & (df_train['dbytes'] != 0)]
else:
    print("Columns 'sbytes' and/or 'dbytes' do not exist in the DataFrame.")
    df_cleaned = df_train

# Menampilkan 5 baris pertama setelah penanganan missing data
print("Data Setelah Penanganan Missing Values:")
print(df_cleaned.head(16))

Columns 'sbytes' and/or 'dbytes' do not exist in the DataFrame.
Data Setelah Penanganan Missing Values:
      id     FILENAME                                                URL  \
0    1.0   100000.txt                          https://www.northcm.ac.th   
1    4.0  8135291.txt                                http://uqr.to/1il1z   
2    5.0   586561.txt               https://www.woolworthsrewards.com.au   
3    6.0   100000.txt  http://test-mantenimiento-bancaweb.azurewebsit...   
4   11.0   412632.txt  http://test-mantenimiento-bancaweb.azurewebsit...   
5   12.0   899992.txt                        https://www.free-marine.com   
6   17.0   100000.txt                           https://www.ridemcts.com   
7   19.0   568276.txt                              https://www.epner.com   
8   20.0    52998.txt                              https://www.az511.com   
9   22.0   100000.txt                         https://www.screenbeam.com   
10  23.0   651121.txt                          https://www.h

In [78]:
for col in num_cols:
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Clipping
    df_train[col] = np.where(df_train[col] < lower_bound, lower_bound, df_train[col])
    df_train[col] = np.where(df_train[col] > upper_bound, upper_bound, df_train[col])

    # Imputation
    median_value = df_train[col].median()
    df_train[col] = np.where(df_train[col] < lower_bound, median_value, df_train[col])
    df_train[col] = np.where(df_train[col] > upper_bound, median_value, df_train[col])

    # Transformation
    if skew(df_train[col]) > 1:
        df_train[col] = np.log1p(df_train[col])

print("Data Setelah Penanganan outlier:")
print(df_train.head(16))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_992\3231887953.py:18: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  if skew(df_train[col]) > 1:
C:\Users\ASUS\AppData\Local\Temp\ipykernel_992\3231887953.py:18: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  if skew(df_train[col]) > 1:
C:\Users\ASUS\AppData\Local\Temp\ipykernel_992\3231887953.py:18: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  if skew(df_train[col]) > 1:
C:\Users\ASUS\AppData\Local\Temp\ipykernel_992\3231887953.py:18: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly iden

Data Setelah Penanganan outlier:
      id     FILENAME                                                URL  \
0    1.0   100000.txt                          https://www.northcm.ac.th   
1    4.0  8135291.txt                                http://uqr.to/1il1z   
2    5.0   586561.txt               https://www.woolworthsrewards.com.au   
3    6.0   100000.txt  http://test-mantenimiento-bancaweb.azurewebsit...   
4   11.0   412632.txt  http://test-mantenimiento-bancaweb.azurewebsit...   
5   12.0   899992.txt                        https://www.free-marine.com   
6   17.0   100000.txt                           https://www.ridemcts.com   
7   19.0   568276.txt                              https://www.epner.com   
8   20.0    52998.txt                              https://www.az511.com   
9   22.0   100000.txt                         https://www.screenbeam.com   
10  23.0   651121.txt                          https://www.hellscent.com   
11  25.0   693302.txt           https://www.marcosimonc

In [79]:
duplicates = df_train.duplicated()
print("Jumlah duplikat:", duplicates.sum())
# Menghapus duplikat berdasarkan kolom 'id'
df_cleaned = df_train.drop_duplicates(subset=['id'], keep='first')


Jumlah duplikat: 0


In [80]:
# 1. Scaling Data dengan MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df_train[num_cols])  # Scaling hanya pada kolom numerik
scaled_df = pd.DataFrame(scaled_features, columns=num_cols, index=df_train.index)

# 2. Membuat fitur polinomial dengan derajat 2
poly = PolynomialFeatures(degree=2, include_bias=False)  # degree=2 untuk membuat fitur kuadrat dan interaksi
poly_features = poly.fit_transform(scaled_df)  # Menggunakan fitur yang sudah diskalakan

# Mendapatkan nama-nama kolom dari fitur polinomial
poly_feature_names = poly.get_feature_names_out(input_features=num_cols)

# Membuat DataFrame untuk fitur polinomial
poly_df = pd.DataFrame(poly_features, columns=poly_feature_names)

# 3. Menggabungkan fitur asli yang sudah diskalakan dengan fitur polinomial
df_combined = pd.concat([df_train.reset_index(drop=True), poly_df], axis=1)

# Menampilkan hasil DataFrame gabungan
print(df_combined.head())


     id     FILENAME                                                URL  \
0   1.0   100000.txt                          https://www.northcm.ac.th   
1   4.0  8135291.txt                                http://uqr.to/1il1z   
2   5.0   586561.txt               https://www.woolworthsrewards.com.au   
3   6.0   100000.txt  http://test-mantenimiento-bancaweb.azurewebsit...   
4  11.0   412632.txt  http://test-mantenimiento-bancaweb.azurewebsit...   

   URLLength                        Domain  DomainLength  IsDomainIP  TLD  \
0       24.0             www.northcm.ac.th          17.0         0.0  com   
1       26.0                       ipfs.io          19.0         0.0   to   
2       30.0  www.woolworthsrewards.com.au          27.0         0.0   au   
3       30.0                       ipfs.io          19.0         0.0  com   
4       26.0        www.nyprowrestling.com          22.0         0.0  com   

   CharContinuationRate  TLDLegitimateProb  ...  NoOfSelfRef^2  \
0                   

In [81]:

class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaler_type='minmax'):
        # Menyimpan jenis scaler yang digunakan
        self.scaler_type = scaler_type
        # Pilih scaler berdasarkan input
        if self.scaler_type == 'minmax':
            self.scaler = MinMaxScaler()
        elif self.scaler_type == 'standard':
            self.scaler = StandardScaler()
        else:
            raise ValueError("scaler_type harus 'minmax' atau 'standard'")

    def fit(self, X, y=None):
        # Melakukan fitting pada data
        self.scaler.fit(X)
        return self

    def transform(self, X):
        # Melakukan transformasi pada data yang sudah difit
        X_scaled = self.scaler.transform(X)
        # Mengembalikan hasil dalam format DataFrame dengan nama kolom yang sama dengan X
        return pd.DataFrame(X_scaled, columns=X.columns)


In [82]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoding_type='onehot'):
        self.encoding_type = encoding_type
        self.encoder = None

    def fit(self, X, y=None):
        if self.encoding_type == 'onehot':
            self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            self.encoder.fit(X)# Changed sparse to sparse_output
        elif self.encoding_type == 'label':
             self.encoder = {col: LabelEncoder().fit(X[col]) for col in X.columns}
        return self

    def transform(self, X):
        if self.encoding_type == 'onehot':
            # Menerapkan OneHotEncoder dan mengembalikan DataFrame
            encoded_array = self.encoder.transform(X)
            columns = self.encoder.get_feature_names_out(input_features=X.columns)
            return pd.DataFrame(encoded_array, columns=columns, index=X.index)

        elif self.encoding_type == 'label':
            # Menerapkan LabelEncoder untuk setiap kolom
            return X.apply(lambda col: self.encoder[col.name].transform(col))


In [83]:
class ImbalanceHandler(BaseEstimator, TransformerMixin):
    def __init__(self, method='smote'):
        if method == 'smote':
            self.handler = SMOTE(random_state=42)
        elif method == 'undersampling':
            self.handler = RandomUnderSampler(random_state=42)
        else:
            raise ValueError("Metode yang valid adalah 'smote' atau 'undersampling'.")

    def fit_resample(self, X, y):
        X_res, y_res = self.handler.fit_resample(X, y)
        print(f"Distribusi kelas setelah penyeimbangan: {Counter(y_res)}") # Fixed indentation
        return X_res, y_res

In [84]:
# from sklearn.pipeline import Pipeline
def create_preprocessing_pipeline(
    numerical_features=None,
    categorical_features=None,
    scaler_type='minmax',
    encoding_type='onehot',
    imbalance_method=None
):
    transformers = []

    # Pipeline untuk fitur numerik
    if numerical_features is not None and len(numerical_features) > 0:
        numerical_pipeline = Pipeline([
            ('scaler', FeatureScaler(scaler_type=scaler_type))
        ])
        transformers.append(('num', numerical_pipeline, numerical_features))

    # Pipeline untuk fitur kategorikal
    if categorical_features is not None and len(categorical_features) > 0:
        categorical_pipeline = Pipeline([
            ('encoder', CategoricalEncoder(encoding_type=encoding_type))
        ])
        transformers.append(('cat', categorical_pipeline, categorical_features))

    # Gabungkan pipeline numerik dan kategorikal
    preprocessor = ColumnTransformer(transformers=transformers)

    # Pipeline akhir
    if imbalance_method is not None:
        imbalance_handler = ImbalanceHandler(method=imbalance_method)
        return Pipeline([
            ('preprocessor', preprocessor),
            ('imbalance_handler', imbalance_handler)
        ])
    else:
        return Pipeline([
            ('preprocessor', preprocessor)
        ])
# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [85]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return predictions

    def _predict(self, x):
         # Menghitung jarak  antara test dengan semua sample training
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]

        # Mendapatkan indeks k tetantanga terdekat
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # Voting mayoritas
        most_common = Counter(k_nearest_labels).most_common()
    
        # Penanganan kasus tie (mengembalikan label terkecil jika seri)
        max_count = most_common[0][1]
        tied_labels = [label for label, count in most_common if count == max_count]
        return min(tied_labels)


In [86]:
# Type your code here

import numpy as np

class GaussianNB:
    def __init__(self):
        self.classes = None  
        self.mean = {}  
        self.var = {}  
        self.priors = {}  

    def fit(self, X, y):
        # Melatih model dengan menghitung mean, variansi, dan prior untuk setiap kelas.
        self.classes = np.unique(y)  
        for cls in self.classes:
            X_cls = X[y == cls] 
            self.mean[cls] = np.mean(X_cls, axis=0)  
            self.var[cls] = np.var(X_cls, axis=0)  
            self.priors[cls] = X_cls.shape[0] / X.shape[0] 

    def _gaussian_pdf(self, class_idx, x):
        #Menghitung PDF Gaussian untuk fitur tertentu.
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def predict(self, X):
        #Memperkirakan label untuk data baru.
        predictions = []
        for x in X:
            posteriors = []
            for cls in self.classes:
                prior = np.log(self.priors[cls]) 
                likelihood = np.sum(np.log(self._gaussian_pdf(cls, x)))  
                posterior = prior + likelihood
                posteriors.append(posterior)
            predictions.append(self.classes[np.argmax(posteriors)])
        return np.array(predictions)


In [87]:
print("Data type of X_train:", X_train.dtypes)
print("Data type of X_val:", X_val.dtypes)


Data type of X_train: FILENAME                        int64
URL                             int64
URLLength                     float64
Domain                          int64
DomainLength                  float64
IsDomainIP                    float64
TLD                             int64
CharContinuationRate          float64
TLDLegitimateProb             float64
URLCharProb                   float64
TLDLength                     float64
NoOfSubDomain                 float64
HasObfuscation                float64
NoOfObfuscatedChar            float64
ObfuscationRatio              float64
NoOfLettersInURL              float64
LetterRatioInURL              float64
NoOfDegitsInURL               float64
DegitRatioInURL               float64
NoOfEqualsInURL               float64
NoOfQMarkInURL                float64
NoOfAmpersandInURL            float64
NoOfOtherSpecialCharsInURL    float64
SpacialCharRatioInURL         float64
IsHTTPS                       float64
LineOfCode                  

In [88]:
print("X_train data types:")
print(X_train.dtypes)

print("\nX_val data types:")
print(X_val.dtypes)



X_train data types:
FILENAME                        int64
URL                             int64
URLLength                     float64
Domain                          int64
DomainLength                  float64
IsDomainIP                    float64
TLD                             int64
CharContinuationRate          float64
TLDLegitimateProb             float64
URLCharProb                   float64
TLDLength                     float64
NoOfSubDomain                 float64
HasObfuscation                float64
NoOfObfuscatedChar            float64
ObfuscationRatio              float64
NoOfLettersInURL              float64
LetterRatioInURL              float64
NoOfDegitsInURL               float64
DegitRatioInURL               float64
NoOfEqualsInURL               float64
NoOfQMarkInURL                float64
NoOfAmpersandInURL            float64
NoOfOtherSpecialCharsInURL    float64
SpacialCharRatioInURL         float64
IsHTTPS                       float64
LineOfCode                    

In [89]:
from sklearn.preprocessing import LabelEncoder

# Apply label encoding to any categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_val[col] = le.transform(X_val[col])




In [90]:
print(X_train.select_dtypes(include='number').head())

       FILENAME    URL  URLLength  Domain  DomainLength  IsDomainIP  TLD  \
28021     82872  95905        NaN   69832          13.0         0.0  130   
71026     82872  96914       25.0   69832          18.0         NaN  130   
89927      4200  13746       25.0   69832          18.0         0.0  497   
94654     29762  56149       27.0   39438          20.0         0.0  497   
35275     82872  72484        NaN   69832          17.0         0.0  345   

       CharContinuationRate  TLDLegitimateProb  URLCharProb  ...  Pay  Crypto  \
28021              0.600000           0.522907          NaN  ...  0.0     NaN   
71026                   NaN                NaN     0.061236  ...  0.0     NaN   
89927              0.818182           0.028555          NaN  ...  NaN     0.0   
94654              1.000000                NaN     0.066401  ...  1.0     0.0   
35275                   NaN                NaN          NaN  ...  1.0     0.0   

       HasCopyrightInfo  NoOfImage  NoOfCSS  NoOfJS  NoO

In [91]:
# If there are columns with NaN values, they should be handled
X_train = X_train.fillna(0)
X_val = X_val.fillna(0)


In [92]:
print(X_train.select_dtypes(include='number').head())

       FILENAME    URL  URLLength  Domain  DomainLength  IsDomainIP  TLD  \
28021     82872  95905        0.0   69832          13.0         0.0  130   
71026     82872  96914       25.0   69832          18.0         0.0  130   
89927      4200  13746       25.0   69832          18.0         0.0  497   
94654     29762  56149       27.0   39438          20.0         0.0  497   
35275     82872  72484        0.0   69832          17.0         0.0  345   

       CharContinuationRate  TLDLegitimateProb  URLCharProb  ...  Pay  Crypto  \
28021              0.600000           0.522907     0.000000  ...  0.0     0.0   
71026              0.000000           0.000000     0.061236  ...  0.0     0.0   
89927              0.818182           0.028555     0.000000  ...  0.0     0.0   
94654              1.000000           0.000000     0.066401  ...  1.0     0.0   
35275              0.000000           0.000000     0.000000  ...  1.0     0.0   

       HasCopyrightInfo  NoOfImage  NoOfCSS  NoOfJS  NoO

In [93]:
X_train = X_train.select_dtypes(include=[np.number])  # Hanya seleksi kolom numerik
X_val = X_val.select_dtypes(include=[np.number])      # Hanya seleksi kolom numerik


In [94]:
from sklearn.metrics import accuracy_score

# 5. Train Model 
knn_model = KNN(k=3)
knn_model.fit(X_train, y_train)

# Predict with the modified dataset
knn_predictions = knn_model.predict(X_val)

gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)

# Predict with GaussianNB
gnb_predictions = gnb_model.predict(X_val)

# Evaluate KNN
knn_accuracy = accuracy_score(y_val, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy:.4f}')

# Evaluate GaussianNB
gnb_accuracy = accuracy_score(y_val, gnb_predictions)
print(f'GaussianNB Accuracy: {gnb_accuracy:.4f}')


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [88]:
X_test = df_test.drop(columns=['id'])
# df_test['label_knn'] = knn_model.predict(X_test)
df_test['label_gnb'] = gnb_model.predict(X_test)

# Select the model with the highest accuracy (optional)
# best_model = knn_model if knn_accuracy > gnb_accuracy else gnb_model

# Create submission file
submission = df_test[['id', 'label']]
submission.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")


TypeError: unsupported operand type(s) for -: 'str' and 'str'